# 02 angr find condition

First we load the compiled binary executable:

In [1]:
import angr, sys

libname = "../build/arm64-v8a/02_angr_find_condition"
project = angr.Project(libname, main_opts={ "base_addr": 0x00200000 })
print(project.arch, hex(project.entry))

<Arch AARCH64 (LE)> 0x20075c


There is something special about libc for android ndk, angr can't handle some functions for us automatically, so we need to handle them manually.

In [2]:
project.hook_symbol("__strncpy_chk", angr.SIM_PROCEDURES['libc']['strncpy']())
project.hook_symbol("__strncpy_chk2", angr.SIM_PROCEDURES['libc']['strncpy']())


WARNING | 2022-05-24 11:10:53,541 | angr.project | Address is already hooked, during hook(0x300020, <SimProcedure strncpy>). Re-hooking.
WARNING | 2022-05-24 11:10:53,543 | angr.project | Address is already hooked, during hook(0x300028, <SimProcedure strncpy>). Re-hooking.


3145768

Now we first find the address of the main function:

In [3]:
main_func = project.loader.find_symbol("main")
print(hex(main_func.rebased_addr))

0x200864


We prepare the state for symbolic execution:

In [4]:
entry_state = project.factory.entry_state(addr=main_func.rebased_addr)
entry_state.options.add(angr.sim_options.ZERO_FILL_UNCONSTRAINED_MEMORY)
entry_state.options.add(angr.sim_options.ZERO_FILL_UNCONSTRAINED_REGISTERS)

Define a function that checks if you have found the state you are looking for.

In [10]:
def is_successful(state):
    # Dump whatever has been printed out by the binary so far into a string.
    stdout_output = state.posix.dumps(1) # stdout

    # Return whether 'Good Job.' has been printed yet.
    # (!)
    return "Good Job.".encode() in stdout_output # :boolean

Same as above, but this time check if the state should abort. If you return False, Angr will continue to step the state. In this specific challenge, the only time at which you will know you should abort is when the program prints "Try again."

In [6]:
def should_abort(state):
    stdout_output = state.posix.dumps(2) # stdout
    return "Try again.".encode() in stdout_output # :boolean

Everything is ready, next we start the symbolic execution to find the path:

In [11]:
simulation = project.factory.simulation_manager(entry_state)
simulation.explore(find=is_successful, avoid=should_abort)
if simulation.found:
    print("Solution(password):", simulation.found[0].posix.dumps(sys.stdin.fileno()).decode())
else:
    print("Cound not find the solution")
    list(map(lambda s: s.callstack.dbg_repr(), simulation.deadended))

sys.stdout.fileno() 55


AttributeError: 'NoneType' object has no attribute 'concretize'